<a href="https://colab.research.google.com/github/vandrearczyk/hecktor-euvip2024/blob/main/baseline_prediction_hecktor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
! pip install scikit-survival

import numpy as np
import pandas as pd
from sksurv.datasets import get_x_y
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.ensemble import RandomSurvivalForest
from sksurv.metrics import concordance_index_censored
from sklearn.model_selection import train_test_split

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.0/49.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 51.8 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.2
    Uninstalling scikit-learn-1.3.2:
      Successfully uninstalled scikit-learn-1.3.2


In [4]:
from google.colab import files

uploaded = files.upload()

Saving features_album_GBM-UPENN-OS_MR_FLAIR-Edema.csv to features_album_GBM-UPENN-OS_MR_FLAIR-Edema.csv
Saving features_album_GBM-UPENN-OS_MR_FLAIR-ET.csv to features_album_GBM-UPENN-OS_MR_FLAIR-ET.csv
Saving features_album_GBM-UPENN-OS_MR_FLAIR-ET-NET.csv to features_album_GBM-UPENN-OS_MR_FLAIR-ET-NET.csv
Saving features_album_GBM-UPENN-OS_MR_FLAIR-ET-NET-Edema.csv to features_album_GBM-UPENN-OS_MR_FLAIR-ET-NET-Edema.csv
Saving features_album_GBM-UPENN-OS_MR_FLAIR-NET.csv to features_album_GBM-UPENN-OS_MR_FLAIR-NET.csv
Saving features_album_GBM-UPENN-OS_MR_T1c-Edema.csv to features_album_GBM-UPENN-OS_MR_T1c-Edema.csv
Saving features_album_GBM-UPENN-OS_MR_T1c-ET.csv to features_album_GBM-UPENN-OS_MR_T1c-ET.csv
Saving features_album_GBM-UPENN-OS_MR_T1c-ET-NET.csv to features_album_GBM-UPENN-OS_MR_T1c-ET-NET.csv
Saving features_album_GBM-UPENN-OS_MR_T1c-ET-NET-Edema.csv to features_album_GBM-UPENN-OS_MR_T1c-ET-NET-Edema.csv
Saving features_album_GBM-UPENN-OS_MR_T1c-NET.csv to features_al

In [9]:
# Combine all uploaded CSV files into one DataFrame
dfs = []
for filename in uploaded.keys():
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)
        dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)
# print(combined_df)

In [10]:
# Keep only the first three columns
first_three_columns = combined_df.iloc[:, :3]

# Keep columns that start with "original_intensity"
original_intensity_columns = combined_df.filter(like="original_intensity")

# Combine the two subsets
filtered_df = pd.concat([first_three_columns, original_intensity_columns], axis=1)

# Print the filtered DataFrame
print(filtered_df.keys())

Index(['PatientID', 'Modality', 'ROI', 'original_intensity_10Percentile',
       'original_intensity_90Percentile', 'original_intensity_Energy',
       'original_intensity_Entropy', 'original_intensity_InterquartileRange',
       'original_intensity_Kurtosis', 'original_intensity_Maximum',
       'original_intensity_Mean', 'original_intensity_MeanAbsoluteDeviation',
       'original_intensity_Median', 'original_intensity_Minimum',
       'original_intensity_Range',
       'original_intensity_RobustMeanAbsoluteDeviation',
       'original_intensity_RootMeanSquared', 'original_intensity_Skewness',
       'original_intensity_TotalEnergy', 'original_intensity_Uniformity',
       'original_intensity_Variance'],
      dtype='object')


In [12]:
# Identify feature columns that start with 'original_intensity_'
feature_columns = [col for col in combined_df.columns if col.startswith('original_intensity_')]

# Melt the DataFrame based on 'PatientID', 'Modality', 'ROI', and feature columns
melted_df = combined_df.melt(id_vars=['PatientID', 'Modality', 'ROI'], value_vars=feature_columns, var_name='Feature')

# Create a new column that combines 'ROI', 'Modality', and 'Feature'
melted_df['Combined'] = melted_df['ROI'] + '_' + melted_df['Modality'] + '_' + melted_df['Feature']

# Pivot the table so that 'Combined' becomes columns, and values are filled in
pivoted_df = melted_df.pivot_table(index='PatientID', columns='Combined', values='value')

# Reset the index to make 'PatientID' a column again
pivoted_df.reset_index(inplace=True)

# Print the resulting DataFrame
# print(pivoted_df)

In [13]:
# Create a Dummy Ground Truth DataFrame for survival prediction
np.random.seed(42)  # For reproducibility
n_samples = len(pivoted_df)

# Randomly generate survival times and status (1 for event occurred, 0 for censored)
survival_times = np.random.uniform(5, 100, size=n_samples)  # Survival time days
survival_status = np.random.randint(0, 2, size=n_samples)   # Binary status

# Add these to the pivoted DataFrame as ground truth
pivoted_df['SurvivalTime'] = survival_times
pivoted_df['SurvivalStatus'] = survival_status

# Prepare the data for training
X = pivoted_df.drop(columns=['PatientID', 'SurvivalTime', 'SurvivalStatus'])
y = np.array([(status, time) for status, time in zip(pivoted_df['SurvivalStatus'], pivoted_df['SurvivalTime'])],
             dtype=[('event', 'bool'), ('time', 'float')])

# Handle missing values by filling them with the mean
X = X.fillna(X.mean())

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a survival model (e.g., Random Survival Forest)
model = RandomSurvivalForest(n_estimators=100, min_samples_split=10, min_samples_leaf=15, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model using Concordance Index
cindex_train = concordance_index_censored(y_train['event'], y_train['time'], model.predict(X_train))[0]
cindex_test = concordance_index_censored(y_test['event'], y_test['time'], model.predict(X_test))[0]

print(f'Concordance Index (Train): {cindex_train:.2f}')
print(f'Concordance Index (Test): {cindex_test:.2f}')

Concordance Index (Train): 0.83
Concordance Index (Test): 0.53
